## Imports

In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search.tool import TavilySearchResults
import sys
# This is a workaround for running the script from the parent directory
sys.path.append('../')

import pysubs2
import json
import os
import pprint

In [7]:
from src.logic.context import gather_context_from_web
from src.logic.context import identify_characters
from src.logic.translate import translate_subs

## Setup required items

In [8]:
json_file = open("../config.json")
json_data = json.load(json_file)
api_key = json_data['openai_api_key']

llm = ChatOpenAI(model="gpt-4o", temperature=json_data['temperature'], api_key=api_key)

os.environ["TAVILY_API_KEY"] = json_data["tavily_api_key"]
search_tool = TavilySearchResults(k=3)

In [9]:
ass_pass = "../sample/sample2.ass"
subs = pysubs2.load(ass_pass)
lines = "\n".join(str(line.text) for line in subs)

## Getting Context

In [10]:
web_search_results = gather_context_from_web(llm, search_tool, output_lang="en",
                                             series_name="too many loosing heroines", keywords="komari nukumizu", transcript=lines)

pprint.pp(web_search_results)

('**Main Characters**: Kazuhiko Nukumizu is a high school student who '
 'considers himself a "background character" and becomes involved in the '
 'romantic dramas of his classmates. Anna Yanami is a popular girl in '
 "Kazuhiko's class who is rejected by her childhood friend Sōsuke Hakamada.\n"
 '\n'
 '**Supporting Characters**: Chika Komari is a first-year student at Tsuwabuki '
 'High School and the Vice-President of the Literature Club. She harbors '
 'romantic feelings for the former Literature Club President, Shintaro Tamaki, '
 'and is close friends with the former Vice-President, Koto Tsukinoki. Lemon '
 'Yakishio is another student who struggles to connect with her childhood '
 'friend Mitsuki Ayano.\n'
 '\n'
 '**Relationships**: Anna Yanami has a complicated relationship with Sōsuke '
 'Hakamada, who chooses Karen Himemiya over her, leaving Anna dejected. '
 'Kazuhiko, despite his lack of romantic experience, becomes entangled in '
 'these relationship dynamics, often witnes

In [12]:
characters_output = identify_characters(llm, input_lang="ja", output_lang="en",
                                        transcript=lines, web_context=web_search_results)

pprint.pp(characters_output)

('- **温水**: Appears to be the narrator and main character, involved in taking '
 "photos for the literature club's homepage. Has a casual tone and interacts "
 'frequently with 小鞠. [Narrative Focus]\n'
 '- **小鞠**: A character who is initially found in the library doing research '
 'for a novel. Has a somewhat defensive and shy demeanor, especially about '
 'being photographed. Engages in banter with 温水 and seems to be knowledgeable '
 'about library use.\n'
 '- **八奈見**: Mentioned in conversation, seems to be a mutual acquaintance of '
 '温水 and 小鞠. Known for eating frequently and being photographed by 温水.')


In [14]:
context_dict = {
    "Characters": characters_output,
    "Web Search Results": web_search_results
    }

context_dict

{'Characters': "- **温水**: Appears to be the narrator and main character, involved in taking photos for the literature club's homepage. Has a casual tone and interacts frequently with 小鞠. [Narrative Focus]\n- **小鞠**: A character who is initially found in the library doing research for a novel. Has a somewhat defensive and shy demeanor, especially about being photographed. Engages in banter with 温水 and seems to be knowledgeable about library use.\n- **八奈見**: Mentioned in conversation, seems to be a mutual acquaintance of 温水 and 小鞠. Known for eating frequently and being photographed by 温水.",
 'Web Search Results': '**Main Characters**: Kazuhiko Nukumizu is a high school student who considers himself a "background character" and becomes involved in the romantic dramas of his classmates. Anna Yanami is a popular girl in Kazuhiko\'s class who is rejected by her childhood friend Sōsuke Hakamada.\n\n**Supporting Characters**: Chika Komari is a first-year student at Tsuwabuki High School and th

## Translating the file

In [15]:
subs = translate_subs(llm, subs, context_dict, 3)

subs.save("temp.ass")

Translating Lines: 100%|██████████| 284/284 [05:44<00:00,  1.21s/line]
